# Đồ án cuối kì Lập trình cho khoa học dữ liệu


## Thành viên nhóm


| |Họ và tên|MSSV|
|:--:|:--|:--:|
|1|Võ Thành Nam|19120301|
|2|Phạm Lưu Mỹ Phúc|19120331|

## Môi trường code

In [1]:
import sys
sys.executable

'C:\\Users\\Nam\\anaconda3\\envs\\min_ds-env\\python.exe'

## Tổng quan về dữ liệu sẽ được dùng để phân tích

Bộ dữ liệu được sử dụng là file `DataScientist.csv`. Đây là dữ liệu nằm trong một bộ dữ liệu gồm nhiều file khác về thông tin tuyển dụng của những ngành nghề liên quan đến Data. 
- Bộ dữ liệu được tác giả *picklesueat* lấy từ trang web https://www.glassdoor.com/ (một trang web tìm kiếm việc làm), sử dụng công cụ `selenium`. 
- Bộ dữ liệu đã được tác giả public tại [Github](https://github.com/picklesueat/data_jobs_data) và được reup tại [Kaggle](https://www.kaggle.com/andrewmvd/data-scientist-jobs).
- Về license của dữ liệu, tác giả không đề cập cụ thể mà cho phép mọi người có thể tải xuống tùy ý.
- File dữ liệu `DataScientist.csv` là file dữ liệu về thông tin tuyển dụng của nghề Data Scientist bao gồm tên vị trí tuyển dụng, mức lương, tên công ty,...

## Import các thư viện cần thiết

In [37]:
import pandas as pd
import numpy as np
import re
import matplotlib as plt
import math

## Khám phá dữ liệu

### Đọc dữ liệu vào từ file csv vào dataframe

In [3]:
ds_df = pd.read_csv('DataScientist.csv')
ds_df.head()

,Unnamed: 0,index,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,Easy Apply
0,0,0,Senior Data Scientist,$111K-$181K (Glassdoor est.),"ABOUT HOPPER\n\nAt Hopper, we’re on a mission ...",3.5,Hopper\n3.5,"New York, NY","Montreal, Canada",501 to 1000 employees,2007,Company - Private,Travel Agencies,Travel & Tourism,Unknown / Non-Applicable,-1,-1
1,1,1,"Data Scientist, Product Analytics",$111K-$181K (Glassdoor est.),"At Noom, we use scientifically proven methods ...",4.5,Noom US\n4.5,"New York, NY","New York, NY",1001 to 5000 employees,2008,Company - Private,"Health, Beauty, & Fitness",Consumer Services,Unknown / Non-Applicable,-1,-1
2,2,2,Data Science Manager,$111K-$181K (Glassdoor est.),Decode_M\n\nhttps://www.decode-m.com/\n\nData ...,-1.0,Decode_M,"New York, NY","New York, NY",1 to 50 employees,-1,Unknown,-1,-1,Unknown / Non-Applicable,-1,True
3,3,3,Data Analyst,$111K-$181K (Glassdoor est.),Sapphire Digital seeks a dynamic and driven mi...,3.4,Sapphire Digital\n3.4,"Lyndhurst, NJ","Lyndhurst, NJ",201 to 500 employees,2019,Company - Private,Internet,Information Technology,Unknown / Non-Applicable,"Zocdoc, Healthgrades",-1
4,4,4,"Director, Data Science",$111K-$181K (Glassdoor est.),"Director, Data Science - (200537)\nDescription...",3.4,United Entertainment Group\n3.4,"New York, NY","New York, NY",51 to 200 employees,2007,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"BBDO, Grey Group, Droga5",-1


Đầu tiên, ta thấy dữ liệu có đến 2 cột index, ta sẽ không sử dụng 2 cột này, do đó sẽ loại bỏ 2 cột này đi.

In [4]:
ds_df.drop(columns = ['Unnamed: 0','index'],inplace = True)

### Số lượng dòng và cột của dữ liệu

In [5]:
n_rows = ds_df.shape[0]
n_cols = ds_df.shape[1]
print(f'Number of rows: {n_rows}')
print(f'Number of columns: {n_cols}')

Number of rows: 3909
Number of columns: 15


### Xét xem các dữ liệu có trùng nhau hay không

In [6]:
print(f'Number of duplicated rows: {sum(ds_df.duplicated())}')

Number of duplicated rows: 0


### Ý nghĩa của các cột trong dataframe
(Dựa theo mô tả tại trang web [Kaggle](https://www.kaggle.com/andrewmvd/data-scientist-jobs))

- **Job Title**: tên vị trí tuyển dụng.
- **Salary Estimate**: mức lương dự kiến.
- **Job Description**: mô tả công việc.
- **Rating**: điểm đánh giá.
- **Company Name**: tên công ty.
- **Location**: địa điểm làm việc.
- **Headquarters**: trụ sở công ty.
- **Size**: kích thước công ty (số lượng nhân viên).
- **Founded**: năm thành lập công ty.
- **Type of ownership**: loại hình sở hữu của công ty.
- **Industry**: phân loại ngành của công ty.
- **Sector**: lĩnh vực của công ty.
- **Revenue**: doanh thu của công ty (theo năm).
- **Competitors**: công ty đối thủ.
- **Easy Apply**: công ty có dễ apply hay không.

### Sơ lược về dữ liệu trong các cột

In [7]:
ds_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3909 entries, 0 to 3908
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Job Title          3909 non-null   object 
 1   Salary Estimate    3909 non-null   object 
 2   Job Description    3909 non-null   object 
 3   Rating             3909 non-null   float64
 4   Company Name       3909 non-null   object 
 5   Location           3909 non-null   object 
 6   Headquarters       3909 non-null   object 
 7   Size               3909 non-null   object 
 8   Founded            3909 non-null   int64  
 9   Type of ownership  3909 non-null   object 
 10  Industry           3909 non-null   object 
 11  Sector             3909 non-null   object 
 12  Revenue            3909 non-null   object 
 13  Competitors        3909 non-null   object 
 14  Easy Apply         3909 non-null   object 
dtypes: float64(1), int64(1), object(13)
memory usage: 458.2+ KB


### Nhận xét chung

- Như ta có thể thấy, dữ liệu không có bất kì dòng nào thiếu hay trùng lắp. Tuy nhiên, khi quan sát bảng dữ liệu ở phía trên, có thế thấy tác giả đã thay thế các dữ liệu (có thể là bị thiếu) bằng giá trị -1.
- Những cột đáng lẽ cần phải ở kiểu dữ liệu là số như `Salary Estimate`, `Size` thì lại có kiểu dữ liệu là object, ta cần phải xử lí những cột này.
- Cột `Company Name` có vẻ bị lỗi khi thu thập dữ liệu (tên công ty gắn liền với rating).
- Cột `Easy Apply` chỉ nên có giá trị `True` hoặc `False`.
- ...

Ta không quan sát được toàn bộ dữ liệu, do đó, ta sẽ phân tích và tiền xử lí cho từng cột trong phần tiền xử lí dữ liệu.

## Tiền xử lí dữ liệu

### Các giá trị của Job title liệu đã đúng hay chưa?

Quan sát các dòng đầu tiên, ta thấy `Job Title` không chỉ bao gồm một vị trí mà có thế có nhiều vị trí, ta sẽ xem xét các giá trị trong cột này.

In [8]:
ds_df['Job Title'].nunique()

2079

Số lượng các `Job Title` khác nhau là rất lớn, do đó ta sẽ xem qua 20 giá trị xuất hiện nhiều nhất và 20 giá trị xuất hiện ít nhất.

In [9]:
job_title_first_20 = ds_df['Job Title'].value_counts()[:20].reset_index()
job_title_last_20 = ds_df['Job Title'].value_counts()[-20:].reset_index()
pd.concat([job_title_first_20,job_title_last_20],axis=1).rename(columns = {'Job Title':'count','index':'Job Title'})

,Job Title,count,Job Title,count
0,Data Scientist,274,Machine Learning-Computer Vision Research Scie...,1
1,Data Engineer,260,Radar Exploitation Scientist (Must Have Top Se...,1
2,Data Analyst,246,+ Data Engineer,1
3,Senior Data Scientist,91,Data Scientist - Power BI,1
4,Senior Data Analyst,47,"Clinical Scientist, Neuroscience (Director level)",1
5,Machine Learning Engineer,47,Experienced Food Scientist / Product Developme...,1
6,Big Data Engineer,41,BI Data Analyst and Specialist,1
7,Business Intelligence Analyst,39,Gilbert Data Science Tutor Jobs,1
8,Senior Data Engineer,37,Program Data Analyst,1
9,Lead Data Scientist,23,"Data Modeler, JLL Technologies",1


Ta có nhận xét: mỗi công ty khi đăng tin tuyển dụng sẽ có một cách thông tin về **Job title** riêng (có thể là tuyển dụng cho nhiều vị trí cùng lúc, ghi tên vị trí cùng với thời gian làm việc,...) nên số lượng giá trị là rất lớn. Tuy nhiên cũng có nhiều vị trí trùng nhau nhưng lại có cách viết khác nhau (Sr., Senior), ta sẽ xử lí những giá trị này.

In [10]:
ds_df['Job Title'] = ds_df['Job Title'].str.replace('Sr.','Senior',regex=False)
ds_df['Job Title'] = ds_df['Job Title'].str.replace('Jr.','Junior',regex=False)

### Loại bỏ rating trong Company name

In [11]:
ds_df['Company Name'] = ds_df['Company Name'].str.split('\n').str[0]

### Tách tên thành phố trong cột Headquarters và xử lí missing value

In [12]:
ds_df['Headquarters'].replace('-1',np.nan,inplace = True)

In [13]:
headquarters_splited = ds_df['Headquarters'].str.split(',',expand=True)
headquarters_splited

,0,1,2
0,Montreal,Canada,None
1,New York,NY,None
2,New York,NY,None
3,Lyndhurst,NJ,None
4,New York,NY,None
...,...,...,...
3904,Irving,TX,None
3905,Woodridge,IL,None
3906,Irving,TX,None
3907,Omaha,NE,None


Một số tên công ty có định dạng thông tin `Headquarters` khác với các công ty còn lại, ta sẽ xem xét và giải quyết những giá trị đặc biệt này.

In [14]:
headquarters_splited[~headquarters_splited[2].isnull()]

,0,1,2
142,New York,NY (US),NY
303,New York,NY (US),NY
1686,Philadelphia,PA (US),PA
1751,Philadelphia,PA (US),PA
1816,New York,NY (US),NY
2118,San Marcos,CA (US),CA


Như vậy, các định dạng khác thường là do một số công ty có ghi thêm quốc gia, ta sẽ loại bỏ các giá trị này.

In [15]:
headquarters_splited[1] = headquarters_splited[1].str.replace('\(\w*\)','',regex=True) 
headquarters_splited.drop(columns=[2],inplace=True)

In [16]:
ds_df[['Headquarters_City','Headquarters_State/Country']] = headquarters_splited
ds_df.drop(columns=['Headquarters'],inplace = True)

### Tiền xử lí các cột Founded

In [17]:
ds_df['Founded'].value_counts().sort_index()

-1       977
 1625      2
 1682      1
 1740      9
 1775     13
        ... 
 2016     49
 2017     40
 2018     27
 2019     14
 2020      1
Name: Founded, Length: 173, dtype: int64

Cột `Founded` có khá nhiều giá trị thiếu, tác giả đã đánh dấu bằng giá trị -1. Ta không có cách nào đễ xác định năm thành lập của những công ty này. Do đó, để tiện cho việc tính toán các tác vụ về sau (tính giá trị thiếu,...), ta sẽ chuyển các giá trị -1 về NaN.

In [18]:
ds_df['Founded'].replace(-1,np.nan,inplace = True)

### Tiền xử lí cột Industry

In [19]:
ds_df['Industry'].value_counts()

-1                                      546
IT Services                             472
Staffing & Outsourcing                  313
Biotech & Pharmaceuticals               293
Computer Hardware & Software            264
                                       ... 
Telecommunications Manufacturing          1
Beauty & Personal Accessories Stores      1
Truck Rental & Leasing                    1
Sporting Goods Stores                     1
Health Fundraising Organizations          1
Name: Industry, Length: 96, dtype: int64

Tương tự cột `Founded`, cột `Industry` cũng có các giá trị thiếu được đánh dấu bằng giá trị `-1`. Ta sẽ chuyển giá trị này thành `NaN`.

In [20]:
ds_df['Industry'].replace('-1',np.nan,inplace = True)

### Tiền xử lí cột Revenue

In [21]:
ds_df['Revenue'].value_counts()

Unknown / Non-Applicable            1163
$10+ billion (USD)                   599
$100 to $500 million (USD)           338
-1                                   229
$50 to $100 million (USD)            214
$2 to $5 billion (USD)               211
$10 to $25 million (USD)             191
$1 to $2 billion (USD)               180
$1 to $5 million (USD)               170
$25 to $50 million (USD)             143
$5 to $10 billion (USD)              133
Less than $1 million (USD)           124
$500 million to $1 billion (USD)     114
$5 to $10 million (USD)              100
Name: Revenue, dtype: int64

Như có thể quan sát, cột này có 3 kiểu thể hiện dữ liệu: giá trị thiếu đánh dấu bằng `-1`, giá trị `Unknown / Non-Applicable` và giá trị có định dạng là một khoảng ước lượng. Ta sẽ tiền xử lí cột này như sau:
- Các dòng có giá trị `-1` hoặc `Unknown / Non-Applicable` sẽ chuyển thành dạng `NaN`.
- Các dòng có giá trị là môt khoảng ước lượng sẽ chuyển thành giá trị trung bình của khoảng. Với 2 khoảng đầu cuối, ta sẽ mặc định thành giá trị phân chia của khoảng đó. Ngoài ra, các giá trị ***billion*** cũng sẽ được đổi thành ***million***.

In [22]:
def convert_to_num(s):
    if type(s) != str:
        return
    if re.compile(r'\$\d+\+ billion').match(s):
        value_range = [int(d) for d in re.findall(r'\d+',s)]
        return sum(value_range)/len(value_range)*1000
    elif re.compile(r'\$\d+ to \$\d+ billion').match(s):
        value_range = [int(d) for d in re.findall(r'\d+',s)]
        return sum(value_range)/len(value_range)*1000
    elif re.compile(r'\$\d+ million to \$\d+ billion').match(s):
        match = re.compile(r'\$(\d+) million to \$(\d+) billion').match(s)
        return (int(match.group(1))+int(match.group(2))*1000)/2
    else:
        value_range = [int(d) for d in re.findall(r'\d+',s)]
        return sum(value_range)/len(value_range)
ds_df['Revenue'].replace('-1',np.nan,inplace = True)
ds_df['Revenue'].replace('Unknown / Non-Applicable',np.nan,inplace = True)
ds_df['Revenue'] = ds_df['Revenue'].apply(convert_to_num)

### Tiền xử lí cột Easy Apply

Ta xem xét kiểu dữ liệu và giá trị trong cột.

In [23]:
print(ds_df['Easy Apply'].dtype)
ds_df['Easy Apply'].value_counts()

object


-1      3745
True     164
Name: Easy Apply, dtype: int64

Các giá trị của cột này nên có dạng `True` hoặc `False`. Các giá trị `-1` trong trường hợp này không phải là giá trị thiếu, mà là công ty không có phương thức `Easy Apply` nên sẽ không có giá trị này. Ta sẽ chuyển các giá trị `-1` thành `False`.

In [24]:
ds_df['Easy Apply'].replace('-1',False,inplace = True)
ds_df['Easy Apply']  = ds_df['Easy Apply'].astype(bool)

### Tiền xử lí cột Salary Estimate

Cột này bao gồm các giá trị là khoảng ước tính, do đó ta sẽ tách thành 2 cột min và max.

In [25]:
def split_to_minmax(s):
    value_range = [int(d) for d in re.findall(r'\d+',s)]
    return value_range
ds_df[['SalaryMin','SalaryMax']] = pd.DataFrame(ds_df['Salary Estimate'].apply(split_to_minmax).to_list())
ds_df.drop(columns=['Salary Estimate'],inplace = True)

### Tiền xử lí cột Rating

Cột `Rating` có khá nhiều giá trị thiếu được đánh dấu bằng giá trị `-1`. Ta sẽ thay thế các giá trị này bằng trung bình của các giá trị còn lại.

In [26]:
ds_df['Rating'].replace(-1,np.nan,inplace = True)
mean = ds_df['Rating'].mean(skipna = True).round(1)
ds_df['Rating'].replace(np.nan,mean,inplace = True)

### Tiền xử lí cột Location va loại bỏ missing value

Trong cột `Location`, mỗi giá trị bao gồm tên thành phố và tên bang cách nhau bởi dấu `,`. Ta sẽ tách các giá trị này thành 2 cột là `City` và `State`.

In [27]:
ds_df['Location'].replace('-1',np.nan,inplace = True)

In [28]:
ds_df[['City','State']] = ds_df['Location'].str.split(',',expand=True)
ds_df.drop(columns = ['Location'], inplace = True)

### Tiền xử lí cột Size

Cột size gồm 3 kiểu thể hiện dữ liệu: gồm `-1` cho các giá trị thiếu, `Unknown` là các giá trị mà công ty không cung cấp và còn lại là các khoảng. Ta sẽ tiền xử lí cột này như sau:
- Các giá trị `-1` và `Unknown` sẽ thay thế bằng giá trị NaN.
- Các giá trị là các khoảng, ta sẽ chuyển thành giá trị trung bình, riêng khoảng lớn nhất sẽ dùng giá trị phân chia.

In [29]:
ds_df['Size'].value_counts()

10000+ employees           993
51 to 200 employees        563
1001 to 5000 employees     553
1 to 50 employees          550
201 to 500 employees       418
501 to 1000 employees      307
-1                         229
5001 to 10000 employees    219
Unknown                     77
Name: Size, dtype: int64

In [39]:
def convert_to_mean(s):
    if type(s) != str:
        return
    value_range = [int(d) for d in re.findall(r'\d+',s)]
    return math.floor(sum(value_range)/len(value_range))
ds_df['Size'].replace('-1',np.nan,inplace = True)
ds_df['Size'].replace('Unknown',np.nan,inplace = True)
ds_df['Size']=ds_df['Size'].apply(convert_to_mean)

### Tiền xử lí các cột Type of ownership, Sector, Competitors

In [42]:
ds_df['Type of ownership'].unique()

array(['Company - Private', 'Unknown', 'Company - Public',
       'Nonprofit Organization', '-1', 'Government', 'Hospital',
       'Self-employed', 'Subsidiary or Business Segment', 'Contract',
       'School / School District', 'Private Practice / Firm',
       'College / University', 'Franchise', 'Other Organization'],
      dtype=object)

In [43]:
ds_df['Sector'].unique()

array(['Travel & Tourism', 'Consumer Services', '-1',
       'Information Technology', 'Business Services', 'Insurance',
       'Finance', 'Retail', 'Media', 'Restaurants, Bars & Food Services',
       'Agriculture & Forestry', 'Non-Profit', 'Education', 'Government',
       'Health Care', 'Oil, Gas, Energy & Utilities',
       'Accounting & Legal', 'Manufacturing', 'Real Estate',
       'Biotech & Pharmaceuticals', 'Arts, Entertainment & Recreation',
       'Aerospace & Defense', 'Construction, Repair & Maintenance',
       'Transportation & Logistics', 'Telecommunications',
       'Mining & Metals'], dtype=object)

In [45]:
ds_df['Competitors'].value_counts()

-1                                                      2760
Amazon, Accenture, Microsoft                              61
Google, Microsoft, Samsung Electronics                    60
Google, Microsoft, Walmart                                47
Google, Microsoft, Apple                                  25
                                                        ... 
Petco, Walmart, Target                                     1
WesternGeco, Petroleum Geo-Services, TGS                   1
Agilent Technologies                                       1
Accenture, Capgemini                                       1
Converge, Future Electronics, Advanced MP Technology       1
Name: Competitors, Length: 422, dtype: int64

Với cả 3 cột trên, ta sẽ tiền xử lí bằng cách thay thế `-1` thành `NaN`.

In [48]:
ds_df[['Type of ownership','Sector','Competitors']] = ds_df[['Type of ownership','Sector','Competitors']].replace('-1',np.nan)

## Đưa ra các câu hỏi cần trả lời

Câu 1

Câu 2

Câu 3

### Câu hỏi 1:

### Câu hỏi 2:

### Câu hỏi 3:

## Tổng kết

## Tài liệu tham khảo